## scikeras Keras regressor integration

The integration of KerasRegressor in pipeline is limited. To use full keras capabilities, you may need to modify the pipeline.

In [ ]:
# Standard loading and preprocessing code

from pinard import utils
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
import random
from pinard import preprocessing as pp
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Concatenate,GlobalAveragePooling1D, Conv1D, Activation, SpatialDropout1D,SeparableConv1D,Flatten, Dropout, Input, MaxPooling1D, DepthwiseConv1D, BatchNormalization, AveragePooling1D
from typing import Dict, Iterable, Any
from tensorflow.keras.optimizers import Adam, Adadelta, SGD
import models
from tensorflow_addons.optimizers import CyclicalLearningRate, LazyAdam


from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
import datetime
from pinard.sklearn import FeatureAugmentation

from scikeras.wrappers import KerasRegressor
# tf.keras.wrappers.scikit_learn.KerasRegressor

import math


# Init basic random
rd_seed = 45678
np.random.seed(rd_seed)
tf.random.set_seed(rd_seed)
random.seed(rd_seed)

preprocessing = [   ('id', pp.IdentityTransformer()),
                    ('baseline', pp.StandardNormalVariate()),
                    ('savgol', pp.SavitzkyGolay()),
                    ('gaussian1', pp.Gaussian(order = 1, sigma = 2)),
                    ('gaussian2', pp.Gaussian(order = 2, sigma = 1)),
                    ('haar', pp.Wavelet('haar')),
                    ('coif3', pp.Wavelet('coif3')),
                    ('detrend', pp.Detrend()),
                    ('msc', pp.MultiplicativeScatterCorrection(scale=False)),
                    ('dv1', pp.Derivate(1,1)),
                    ('dv2', pp.Derivate(2,1)),
                    ('dv3', pp.Derivate(2,2)),
                    
                    ('baseline*savgol', Pipeline([('_sg1',pp.StandardNormalVariate()),('_sg2',pp.SavitzkyGolay())])),
                    ('baseline*gaussian1', Pipeline([('_sg1',pp.StandardNormalVariate()),('g2', pp.Gaussian(order = 1, sigma = 2) )])),
                    ('baseline*gaussian2', Pipeline([('_sg1',pp.StandardNormalVariate()),('g2', pp.Gaussian(order = 2, sigma = 1) )])),
                    ('baseline*haar', Pipeline([('_sg1',pp.StandardNormalVariate()),('_sg2',pp.Wavelet('haar'))])),
                    
                    # ('savgol*savgol', Pipeline([('_sg1',pp.SavitzkyGolay()),('_sg2',pp.SavitzkyGolay())])),
                    # ('savgol*baseline', Pipeline([('_sg1',pp.SavitzkyGolay()),('_sg2',pp.StandardNormalVariate())])),
                    ('savgol*gaussian1', Pipeline([('_sg1',pp.SavitzkyGolay()),('g2', pp.Gaussian(order = 1, sigma = 2) )])),
                    ('savgol*gaussian2', Pipeline([('_sg1',pp.SavitzkyGolay()),('g2', pp.Gaussian(order = 2, sigma = 1) )])),
                    # ('savgol*haar', Pipeline([('_sg1',pp.SavitzkyGolay()), ('haar', pp.Wavelet('haar'))])),
                    # ('gaussian1*savgol', Pipeline([('_g1',pp.Gaussian(order = 1, sigma = 2)),('_sg3',pp.SavitzkyGolay())])),
                    ('gaussian2*savgol', Pipeline([('_g2',pp.Gaussian(order = 1, sigma = 2)),('_sg4',pp.SavitzkyGolay())])),
                    # ('haar*savgol', Pipeline([('_haar2',pp.Wavelet('haar')),('_sg5',pp.SavitzkyGolay())])),
                    # ('haar*gaussian1', Pipeline([('_haar2',pp.Wavelet('haar')), ('g2', pp.Gaussian(order = 1, sigma = 2)) ])),
                    ('haar*gaussian2', Pipeline([('_haar2',pp.Wavelet('haar')), ('g2', pp.Gaussian(order = 2, sigma = 1)) ])),
                ]




# custom Keras Callback that store 
class Auto_Save(Callback):
    best_weights = []
    def __init__(self):
        super(Auto_Save, self).__init__()
        self.best = np.Inf
                
    def on_epoch_end(self, epoch, logs=None):
        current_loss = logs.get('val_loss')
        if np.less(current_loss, self.best):
            self.best = current_loss            
            Auto_Save.best_weights = self.model.get_weights()
            print("Best so far >", self.best)
            
    def on_train_end(self, logs=None):
        if self.params['verbose'] == 2:
            print('\nSaved best {0:6.4f}\n'.format(self.best))


class Print_LR(Callback):    
    def on_epoch_end(self, epoch, logs=None):
        iteration = self.model.optimizer.iterations.numpy()
        # lr = clr(iteration).numpy()
        lr = self.model.optimizer.learning_rate
        if self.params['verbose'] == 2:
            print("Iteration {} - Learning rate: {}".format(iteration, lr) )

# def cyclic_optimizer(X_train, BATCH_SIZE, p):
#     MIN_LR, MAX_LR, CYCLE_LENGTH = p['MIN_LR'], p['MAX_LR'], p['CYCLE_LENGTH']
#     steps_per_epoch = len(X_train) // BATCH_SIZE

#     clr = CyclicalLearningRate(
#         initial_learning_rate=MIN_LR,
#         maximal_learning_rate=MAX_LR,
#         scale_fn=lambda x: 1/(2.**(x-1)),
#         step_size= CYCLE_LENGTH * steps_per_epoch
#     )
#     return Adam(clr)


def keras_model(meta):
    # print(input_shape)
    input_shape = meta["X_shape_"][1:]
    # print("---", meta["X_shape_"], input_shape)
    # input_shape = shape[1:]
    # optimizer = meta["opt"]
    # model = models.xception(input_shape)
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(SpatialDropout1D(0.2))
    model.add(DepthwiseConv1D(kernel_size=7, padding="same",depth_multiplier=2, activation='relu'))
    model.add(DepthwiseConv1D(kernel_size=7, padding="same",depth_multiplier=2, activation='relu'))
    model.add(AveragePooling1D(pool_size=2,strides=2))
    model.add(BatchNormalization())
    model.add(DepthwiseConv1D(kernel_size=5, padding="same",depth_multiplier=2, activation='relu'))
    model.add(DepthwiseConv1D(kernel_size=5, padding="same",depth_multiplier=2, activation='relu'))
    model.add(AveragePooling1D(pool_size=2,strides=2))
    model.add(BatchNormalization())
    model.add(DepthwiseConv1D(kernel_size=9, padding="same",depth_multiplier=2, activation='relu'))
    model.add(DepthwiseConv1D(kernel_size=9, padding="same",depth_multiplier=2, activation='relu'))
    model.add(MaxPooling1D(pool_size=2,strides=2))
    model.add(BatchNormalization())
    
    model.add(SeparableConv1D(128, kernel_size=5, depth_multiplier=1, padding="same", activation='relu'))
    model.add(Conv1D(filters=32, kernel_size=3, padding="same"))
    model.add(MaxPooling1D(pool_size=5,strides=3))
    model.add(SpatialDropout1D(0.2))
    model.add(Flatten())
    model.add(Dense(units=128, activation="sigmoid"))
    model.add(Dense(units=32, activation="sigmoid"))
    model.add(Dropout(0.1))
    model.add(Dense(units=1, activation="sigmoid"))
    # # optimizer = Adadelta(learning_rate=1.0, rho=0.95, epsilon=1e-07)
    # model.compile(loss = 'mean_squared_error', metrics=['mse'], optimizer = optimizer)
    # model.compile(loss = 'mean_squared_error', metrics=['mse'], optimizer = "adam")
    # model.summary()
    return model


def train_model(Xcal, Ycal, Xval, Yval, BATCH_SIZE, EPOCH, cycle_params):  
    X_train, y_train = utils.load(Xcal, Ycal, y_cols=1)
    X_test, y_test = utils.load(Xval, Yval, y_cols=1)
    
    y_scaler = MinMaxScaler()
    y_scaler.fit(y_train.reshape((-1,1)))
    y_valid = y_scaler.transform(y_test.reshape((-1,1)))
  
    transformer_pipeline = Pipeline([
        ('scaler', MinMaxScaler()), 
        ('preprocessing', FeatureAugmentation(preprocessing)), 
    ])
    
    transformer_pipeline.fit(X_train)
    X_valid = transformer_pipeline.transform(X_test)
    early_stop = EarlyStopping(monitor='val_loss', patience=400, verbose=0, mode='min') 
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    # reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=100, verbose=1, min_delta=0.5e-5, mode='min')

    def scale_fn(x):
        return 1 / (2.0 ** (x - 1))
        # return 1. ** x
        
    MIN_LR, MAX_LR, CYCLE_LENGTH = cycle_params['MIN_LR'], cycle_params['MAX_LR'], cycle_params['CYCLE_LENGTH']
    def clr(epoch, lr):
        initial_learning_rate = MIN_LR
        maximal_learning_rate = MAX_LR
        step_size = CYCLE_LENGTH
        step_as_dtype = float(epoch)
        cycle = math.floor(1 + step_as_dtype / (2 * step_size))
        x = abs(step_as_dtype / step_size - 2 * cycle + 1)
        mode_step = cycle # if scale_mode == "cycle" else step
        return initial_learning_rate + ( maximal_learning_rate - initial_learning_rate) * max(0, (1 - x)) * scale_fn(mode_step)

    # clr = CyclicalLearningRate(
    #     initial_learning_rate=MIN_LR,
    #     maximal_learning_rate=MAX_LR,
    #     scale_fn=scale_fn,
    #     step_size= CYCLE_LENGTH * steps_per_epoch
    # )
    
    # kerasModel = keras_model((len(X_train),len(preprocessing)))
    # # print("---", X_train.shape)
    # kerasModel.compile(loss = 'mean_squared_error', metrics=['mse'], optimizer = LazyAdam(0.001))
    
    lrScheduler = tf.keras.callbacks.LearningRateScheduler(clr)
    
    k_regressor = KerasRegressor(model = keras_model,
                                loss = 'mean_squared_error', metrics=['mse'], optimizer = "adam",
                                callbacks=[Auto_Save(), early_stop, lrScheduler, tensorboard_callback],
                                epochs=EPOCH,
                                batch_size=BATCH_SIZE,
                                # scale_fn=scale_fn,
                                fit__validation_data = (X_valid, y_valid),
                                verbose = 2)

    # estimation pipeline
    pipeline = Pipeline([
        ('trans', transformer_pipeline), 
        ('KerasNN', k_regressor)
    ])

    estimator = TransformedTargetRegressor(regressor = pipeline, transformer = y_scaler)
    estimator.fit(X_train, y_train)

    estimator.regressor_[1].model_.set_weights(Auto_Save.best_weights)

    Y_preds = estimator.predict(X_test)
    RMSE = math.sqrt(mean_squared_error(y_test, Y_preds))
    print("MAE", mean_absolute_error(y_test, Y_preds))
    print("MSE", mean_squared_error(y_test, Y_preds))
    print("RMSE", RMSE)
    print("MAPE", mean_absolute_percentage_error(y_test, Y_preds))
    print("R²", r2_score(y_test, Y_preds))
    
    return y_test, Y_preds, RMSE, estimator.regressor_[1].model_

In [ ]:
from pathlib import Path
import glob
rootdir = Path('data')
file_list = [f for f in rootdir.glob('**/*') if f.is_dir()]

BATCH_SIZE = 256
EPOCHS = 10000

cycle_params = {
    'MIN_LR': 1e-5,
    'MAX_LR': 1e-2,
    'CYCLE_LENGTH': 256    
}

for dir in file_list:
    training_name = str(dir)
    projdir = Path(dir)
    Xcal = next(projdir.glob("*Xcal*"))
    Ycal = next(projdir.glob("*Ycal*"))
    Xval = next(projdir.glob("*Xval*"))
    Yval = next(projdir.glob("*Yval*"))
    
    print("*"*15)
    print(str(dir))
    print("-"*15)    
    
    y_test, Y_preds, RMSE, model = train_model(Xcal, Ycal, Xval, Yval, BATCH_SIZE, EPOCHS, cycle_params)

    np.savetxt(training_name + 'res'+str(RMSE)+'.csv', np.column_stack((y_test,Y_preds)))
    model.save(training_name + '_model')
    
    np.savetxt(training_name +'/res'+str(RMSE)+'.csv', np.column_stack((y_test,Y_preds)))
    model.save(training_name + '/model')
